#### 1. 데이터셋 다운로드

In [8]:
# 대체 방법 1: requests 라이브러리 사용
import requests
import os

url = "https://www.dropbox.com/s/5wwskxctvcxiuea/MedNIST.tar.gz?dl=1"  # dl=1 파라미터 추가
response = requests.get(url, stream=True)
with open("MedNIST.tar.gz", "wb") as f:
    for chunk in response.iter_content(chunk_size=8192):
        f.write(chunk)

# 압축 해제는 기존 코드와 동일
import tarfile
datafile = tarfile.open("MedNIST.tar.gz")
datafile.extractall()
datafile.close()

C:\Users\inhag\AppData\Local\Temp\ipykernel_1108\3105613033.py:14: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  datafile.extractall()


#### 2. MONAI 및 필요 모듈 설치, 가져오기

In [2]:
# '-q' 옵션은 설치 시 출력되는 메시지를 최소화합니다.
# MONAI 라이브러리 설치 명령어
# MONAI의 최신 주간 빌드를 설치합니다. 
# 'gdown', 'nibabel', 'tqdm', 'itk' 등 추가 종속성도 함께 설치합니다.
!pip install -q "monai-weekly[gdown, nibabel, tqdm, itk]"
# 필요한 추가 라이브러리도 함께 설치합니다. 'pytorch', 'scikit-learn', 'pillow', 'matplotlib'
!pip install -q numpy scikit-learn pillow matplotlib
# GPU 사용을 위해서 CUDA torch를 설치합니다. 설치한 CUDA 플랫폼에 맞게 선택하여 설치할 수 있도록 합니다. https://pytorch.org/get-started/locally/
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

In [2]:
# 필요한 라이브러리 및 모듈을 가져옵니다
import os  # 운영 체제와 상호작용하기 위한 모듈
import shutil  # 파일 및 디렉토리 조작을 위한 모듈
import tempfile  # 임시 파일 및 디렉토리 생성 모듈
import matplotlib.pyplot as plt  # 데이터 시각화를 위한 matplotlib의 pyplot 모듈
from PIL import Image  # 이미지 작업을 위한 Python Imaging Library
import torch  # PyTorch 딥러닝 프레임워크
import numpy as np  # 수학적 연산을 위한 numpy 라이브러리
from sklearn.metrics import classification_report  # 분류 모델의 성능 평가를 위한 함수

# MONAI 관련 모듈 가져오기
from monai.apps import download_and_extract  # 다운로드 및 압축 해제를 위한 유틸리티 함수
from monai.config import print_config  # MONAI 구성 정보를 출력하는 함수
from monai.metrics import ROCAUCMetric  # ROC AUC 평가 메트릭
from monai.networks.nets import DenseNet121  # DenseNet121 네트워크
from monai.transforms import (  # 데이터 변환 및 전처리 파이프라인 구성 요소들
    Activations,  # 출력값에 활성화 함수를 적용
    EnsureChannelFirst,  # 이미지의 채널 순서를 첫 번째로 변경
    AsDiscrete,  # 출력을 이산 값으로 변환 (예: 클래스화)
    Compose,  # 여러 변환을 순차적으로 적용
    LoadImage,  # 이미지 로드
    RandFlip,  # 무작위로 이미지 뒤집기
    RandRotate,  # 무작위로 이미지 회전
    RandZoom,  # 무작위로 이미지 확대/축소
    ScaleIntensity,  # 이미지의 강도를 스케일링
    ToTensor,  # 데이터를 PyTorch 텐서로 변환
)
from monai.data import Dataset, DataLoader  # 데이터셋과 데이터로더 구성
from monai.utils import set_determinism  # 무작위성 제어를 위한 시드 설정

# MONAI 구성 정보 출력
print_config()
# MONAI와 관련된 버전 및 설정 정보를 출력하여 현재 환경을 확인합니다.

MONAI version: 1.5.dev2512
Numpy version: 2.2.4
Pytorch version: 2.6.0+cu126
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: e4701e24c97d1f8c7ba40777c238cdfe14b04581
MONAI __file__: c:\Users\<username>\.virtualenvs\Chapter5-HHAVn1Ew\Lib\site-packages\monai\__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
ITK version: 5.4.3
Nibabel version: 5.3.2
scikit-image version: NOT INSTALLED or UNKNOWN VERSION.
scipy version: 1.15.2
Pillow version: 11.1.0
Tensorboard version: NOT INSTALLED or UNKNOWN VERSION.
gdown version: 5.2.0
TorchVision version: 0.21.0+cu126
tqdm version: 4.67.1
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 7.0.0
pandas version: NOT INSTALLED or UNKNOWN VERSION.
einops version: NOT INSTALLED or UNKNOWN VERSION.
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
pynrrd version: NOT INSTALLED or UNKNOWN VERSION.
clearml v

#### 3. 데이터셋 폴더에서 이미지 파일 이름 읽기

In [3]:
# 데이터 디렉터리 경로 설정
data_folder = './MedNIST/'
# MedNIST 데이터셋의 경로를 설정합니다.

# 클래스 이름 설정
category_names = sorted([item for item in os.listdir(data_folder) if os.path.isdir(os.path.join(data_folder, item))])
# 데이터 디렉터리 내의 모든 폴더 이름을 가져와, 각 폴더를 클래스 이름으로 사용합니다.
# 각 클래스 이름을 오름차순으로 정렬합니다.

# 클래스 수 계산
category_count = len(category_names)
# 분류할 총 클래스 수를 계산합니다.

# 이미지 파일 경로 수집
img_paths = [[os.path.join(data_folder, category, file) 
             for file in os.listdir(os.path.join(data_folder, category))] 
            for category in category_names]
# 각 클래스 폴더 내의 이미지 파일 경로를 수집합니다.
# 각 클래스 이름에 해당하는 경로를 만든 후, 해당 폴더 내의 모든 파일 경로를 리스트로 생성합니다.

# 이미지 파일과 레이블 리스트 초기화
all_image_paths = []  # 모든 이미지 파일 경로를 담을 리스트
all_image_labels = []  # 각 이미지에 대한 클래스 레이블을 담을 리스트

# 이미지 파일과 레이블 리스트 구성
for idx, category in enumerate(category_names):
   all_image_paths.extend(img_paths[idx])
   # 해당 클래스의 모든 이미지 파일 경로를 리스트에 추가합니다.
   all_image_labels.extend([idx] * len(img_paths[idx]))
   # 해당 클래스의 레이블 값을 이미지 수만큼 추가합니다.

# 총 이미지 수 계산
total_images = len(all_image_labels)
# 전체 이미지 수를 계산합니다.

# 이미지 크기 확인
img_width, img_height = Image.open(all_image_paths[0]).size
# 첫 번째 이미지 파일을 열어, 이미지의 가로 및 세로 크기를 가져옵니다.

#### 4. 학습, 검증, 테스트 데이터 리스트 준비

In [9]:
import random

# 검증 및 테스트 데이터 비율 설정
validation_ratio, testing_ratio = 0.1, 0.1
# 검증 데이터와 테스트 데이터의 비율을 각각 10%로 설정합니다.

# 학습, 검증, 테스트 데이터를 위한 리스트 초기화
train_images, train_labels = [], []  # 학습 데이터 이미지와 레이블 리스트
val_images, val_labels = [], []  # 검증 데이터 이미지와 레이블 리스트
test_images, test_labels = [], []  # 테스트 데이터 이미지와 레이블 리스트

# 데이터셋을 학습, 검증, 테스트 데이터로 분할
for i in range(total_images):
   random_value = np.random.random()
   # 0과 1 사이의 랜덤한 실수 생성
   
   if random_value < validation_ratio:
       # 랜덤 값이 검증 비율보다 작으면 검증 데이터로 할당
       val_images.append(all_image_paths[i])
       val_labels.append(all_image_labels[i])
   elif random_value < testing_ratio + validation_ratio:
       # 랜덤 값이 (검증 비율 + 테스트 비율)보다 작으면 테스트 데이터로 할당
       test_images.append(all_image_paths[i])
       test_labels.append(all_image_labels[i])
   else:
       # 그 외에는 학습 데이터로 할당
       train_images.append(all_image_paths[i])
       train_labels.append(all_image_labels[i])

# 학습 데이터의 10%만 샘플링
sample_size = int(len(train_images) * 0.1)  # 전체 학습 데이터의 10%
indices = random.sample(range(len(train_images)), sample_size)  # 랜덤 인덱스 추출
sampled_train_images = [train_images[i] for i in indices]  # 선택된 이미지
sampled_train_labels = [train_labels[i] for i in indices]  # 선택된 레이블

print(f"원본 학습 데이터 크기: {len(train_images)}, 샘플링된 학습 데이터 크기: {len(sampled_train_images)}")

# 원래 학습 데이터 대신 샘플링된 데이터 사용
train_images = sampled_train_images
train_labels = sampled_train_labels

# 데이터 분할 결과 출력
print("Training count =", len(train_images), "Validation count =", len(val_images), "Test count =", len(test_images))
# 학습, 검증, 테스트 데이터의 개수를 출력합니다.

원본 학습 데이터 크기: 47097, 샘플링된 학습 데이터 크기: 4709
Training count = 4709 Validation count = 5938 Test count = 5919


#### 5. 데이터 전처리를 위한 MONAI 변환, 데이터셋 및 데이터로더 정의

In [10]:
# 학습 데이터 전처리 변환 정의
training_transforms = Compose([
   LoadImage(image_only=True),  # 이미지만 로드 (메타데이터 제외)
   EnsureChannelFirst(),  # 이미지의 채널을 첫 번째 축으로 설정
   ScaleIntensity(),  # 픽셀 값의 강도를 정규화
   RandRotate(range_x=15, prob=0.5, keep_size=True),  # 랜덤으로 이미지 회전 (각도 범위: -15도 ~ 15도, 50% 확률)
   RandFlip(spatial_axis=0, prob=0.5),  # 랜덤으로 이미지 좌우 반전 (50% 확률)
   RandZoom(min_zoom=0.9, max_zoom=1.1, prob=0.5, keep_size=True),  # 랜덤 확대/축소 (0.9배 ~ 1.1배, 50% 확률)
   ToTensor()  # PyTorch 텐서로 변환
])

# 검증 및 테스트 데이터 전처리 변환 정의
validation_transforms = Compose([
   LoadImage(image_only=True),  # 이미지만 로드 (메타데이터 제외)
   EnsureChannelFirst(),  # 이미지의 채널을 첫 번째 축으로 설정
   ScaleIntensity(),  # 픽셀 값의 강도를 정규화
   ToTensor()  # PyTorch 텐서로 변환
])

# 모델의 출력값을 활성화 함수로 처리
activation = Activations(softmax=True)  # 소프트맥스 활성화 함수를 사용하여 확률 값으로 변환
to_onehot = AsDiscrete(to_onehot=category_count)  # 레이블을 원-핫 인코딩 형태로 변환

# MedNISTDataset 클래스 정의
class MedNISTDataset(Dataset):

   def __init__(self, image_files, labels, transforms):
       # 초기화 메서드, 이미지 파일 경로, 레이블, 변환을 인자로 받습니다.
       self.image_files = image_files
       self.labels = labels
       self.transforms = transforms

   def __len__(self):
       # 데이터셋의 총 크기 반환
       return len(self.image_files)

   def __getitem__(self, index):
       # 인덱스에 해당하는 이미지와 레이블을 반환합니다.
       return self.transforms(self.image_files[index]), self.labels[index]

# 학습 데이터셋 및 데이터로더 생성
train_dataset = MedNISTDataset(train_images, train_labels, training_transforms)
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True, num_workers=2)
# 학습 데이터 로더를 생성합니다.
# 배치 크기는 300, 데이터를 셔플하고 2개의 워커를 사용합니다.

# 검증 데이터셋 및 데이터로더 생성
val_dataset = MedNISTDataset(val_images, val_labels, validation_transforms)
val_loader = DataLoader(val_dataset, batch_size=100, num_workers=2)
# 검증 데이터 로더를 생성합니다.
# 배치 크기는 300, 셔플하지 않고 2개의 워커를 사용합니다.

# 테스트 데이터셋 및 데이터로더 생성
test_dataset = MedNISTDataset(test_images, test_labels, validation_transforms)
test_loader = DataLoader(test_dataset, batch_size=100, num_workers=2)
# 테스트 데이터 로더를 생성합니다.
# 배치 크기는 300, 셔플하지 않고 2개의 워커를 사용합니다.

#### 6. 학습 모델 아키텍처와 옵티마이저 정의

In [11]:
# 학습을 수행할 장치 설정
training_device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# GPU를 사용하여 학습을 수행할 수 있도록 장치를 CUDA로 설정하고, GPU가 사용 가능한 상태가 아니라면 CPU로 설정됩니다.
print(training_device)

# DenseNet121 모델 정의
network = DenseNet121(
   spatial_dims=2,  # 입력 이미지의 공간 차원 (2D 이미지)
   in_channels=1,  # 입력 채널 수 (흑백 이미지라 1)
   out_channels=category_count  # 출력 채널 수 (클래스 수)
).to(training_device)
# DenseNet121 모델을 생성하고, 장치에 할당합니다 (GPU 또는 CPU).

# 손실 함수 정의
criterion = torch.nn.CrossEntropyLoss()
# 분류 문제에서 많이 사용하는 크로스 엔트로피 손실 함수를 정의합니다.

# 옵티마이저 정의
optim = torch.optim.Adam(network.parameters(), 1e-5)
# 모델의 파라미터들을 업데이트하기 위해 Adam 옵티마이저를 사용합니다.
# 학습률(learning rate)은 1e-5로 설정하였습니다.

# 학습 반복 횟수 및 검증 간격 설정
num_epochs = 4
# 전체 학습을 수행할 에폭 수를 4로 설정합니다.

validation_interval = 1
# 검증을 수행할 간격을 설정합니다 (여기서는 매 에폭마다 검증을 수행합니다).

cuda:0


#### 7. 모델 훈련

In [ ]:
# 초기 설정 값 정의
highest_metric = -1  # 최고 평가 지표 (accuracy) 초기값 설정
highest_metric_epoch = -1  # 최고 평가 지표를 얻은 에폭 초기값 설정
epoch_loss_history = list()  # 에폭당 손실 값들을 저장할 리스트
auc_evaluator = ROCAUCMetric()  # AUC(Area Under the Curve) 메트릭 정의
metrics_history = list()  # 평가 지표 값들을 저장할 리스트

# 학습 진행 상황 모니터링을 위한 추가 정보
total_train_samples = len(test_dataset)
total_batches = len(test_dataset) // train_loader.batch_size
print(f"총 학습 샘플 수: {total_train_samples}, 배치 크기: {train_loader.batch_size}, 총 배치 수: {total_batches}")

# 학습 시작 시간 기록
import time
start_time = time.time()

# 학습 루프 시작
for epoch in range(num_epochs):
   print('-' * 10)
   print(f"epoch {epoch + 1}/{num_epochs}")  # 현재 에폭 정보 출력
   epoch_start_time = time.time()  # 에폭 시작 시간
   network.train()  # 모델을 학습 모드로 설정
   epoch_loss = 0  # 현재 에폭 손실 초기값
   step = 0  # 학습 스텝 초기화
   correct_train = 0  # 학습 중 정확히 예측한 샘플 수
   total_train = 0  # 학습 중 처리한 총 샘플 수

   # 학습 데이터를 반복하며 학습 수행
   for batch_data in train_loader:
       step += 1
       inputs, labels = batch_data[0].to(training_device), batch_data[1].to(training_device)
       # 배치 데이터를 장치(CPU/GPU)로 전송
       batch_size = inputs.size(0)  # 현재 배치 크기
       total_train += batch_size

       optim.zero_grad()  # 옵티마이저의 기울기 초기화
       outputs = network(inputs)  # 모델에 입력 데이터를 전달하여 예측값 계산
       loss = criterion(outputs, labels)  # 손실 함수 계산
       loss.backward()  # 손실 함수의 기울기 계산 (역전파 수행)
       optim.step()  # 기울기를 사용하여 모델 파라미터 업데이트

       # 학습 중 정확도 계산
       _, predicted = torch.max(outputs.data, 1)
       correct_train += (predicted == labels).sum().item()

       epoch_loss += loss.item()  # 현재 손실 값을 누적
       
       # 진행 상황 출력 (10% 간격 또는 원래 출력)
       if step % max(1, total_batches // 10) == 0:
           current_acc = 100 * correct_train / total_train
           processed_percent = 100 * step / total_batches
           print(f"진행률: {processed_percent:.1f}% ({step}/{total_batches}) | 현재 정확도: {current_acc:.2f}%")
       
       print(f"{step}/{len(test_dataset) // train_loader.batch_size}, train_loss: {loss.item():.4f}")
       # 현재 스텝 및 손실 값을 출력

   epoch_loss /= step  # 에폭 평균 손실 계산
   epoch_loss_history.append(epoch_loss)  # 에폭 손실 값 저장
   
   # 에폭 완료 후 추가 정보
   epoch_acc = 100 * correct_train / total_train
   epoch_time = time.time() - epoch_start_time
   print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")  # 에폭 평균 손실 값 출력
   print(f"epoch {epoch + 1} training accuracy: {epoch_acc:.2f}% | 소요 시간: {epoch_time:.2f}초")

   # 검증 단계
   if (epoch + 1) % validation_interval == 0:
       print("검증 시작...")
       val_start_time = time.time()
       network.eval()  # 모델을 평가 모드로 설정
       with torch.no_grad():  # 평가에서는 기울기 계산을 하지 않음
           y_pred = torch.tensor([], dtype=torch.float32, device=training_device)
           y = torch.tensor([], dtype=torch.long, device=training_device)
           # 예측값과 레이블을 저장할 텐서 초기화

           # 검증 데이터 반복
           for val_step, val_data in enumerate(val_loader, 1):
               val_images, val_labels = val_data[0].to(training_device), val_data[1].to(training_device)
               # 배치 데이터를 장치로 전송
               y_pred = torch.cat([y_pred, network(val_images)], dim=0)
               # 모델에 검증 데이터를 입력하여 예측값을 얻고 y_pred에 누적
               y = torch.cat([y, val_labels], dim=0)
               # 레이블을 y에 누적
               
               # 검증 진행상황 표시
               if val_step % max(1, len(val_loader) // 5) == 0:
                   print(f"  검증 진행률: {100 * val_step / len(val_loader):.1f}% ({val_step}/{len(val_loader)})")

           y_onehot = [to_onehot(i) for i in y]  # 레이블을 원-핫 인코딩 형태로 변환
           y_pred_act = [activation(i) for i in y_pred]  # 예측값에 활성화 함수 적용
           auc_evaluator(y_pred_act, y_onehot)  # AUC 계산
           auc_result = auc_evaluator.aggregate()  # AUC 결과 계산
           auc_evaluator.reset()  # AUC 메트릭 초기화
           del y_pred_act, y_onehot  # 사용 후 텐서 삭제

           metrics_history.append(auc_result)  # AUC 값을 리스트에 저장
           acc_value = torch.eq(y_pred.argmax(dim=1), y)  # 예측값과 실제 값 비교하여 정확도 계산
           acc_metric = acc_value.sum().item() / len(acc_value)  # 정확도 비율 계산
           
           val_time = time.time() - val_start_time

           # 현재 에폭의 정확도가 최고 기록일 경우 모델 저장
           if acc_metric > highest_metric:
               improvement = acc_metric - highest_metric
               highest_metric = acc_metric
               highest_metric_epoch = epoch + 1
               torch.save(network.state_dict(), 'best_metric_model.pth')
               print('saved new best metric model')  # 새로운 최고 정확도 모델 저장
               print(f'성능 개선: +{improvement:.4f}')

           print(f"current epoch: {epoch + 1} current AUC: {auc_result:.4f}"
                 f" current accuracy: {acc_metric:.4f} best AUC: {highest_metric:.4f}"
                 f" at epoch: {highest_metric_epoch}")
           print(f"검증 소요 시간: {val_time:.2f}초")
           # 현재 에폭의 AUC, 정확도, 최고 기록 등을 출력

# 학습 완료 후 최종 출력
total_time = time.time() - start_time
hours, remainder = divmod(total_time, 3600)
minutes, seconds = divmod(remainder, 60)

print(f"train completed, best_metric: {highest_metric:.4f} at epoch: {highest_metric_epoch}")
print(f"총 학습 소요 시간: {int(hours)}시간 {int(minutes)}분 {seconds:.2f}초")
# 학습 완료 메시지와 최고 성능의 에폭 정보 출력

총 학습 샘플 수: 5919, 배치 크기: 100, 총 배치 수: 59
----------
epoch 1/4


#### 8. 손실 및 평가 지표 시각화

In [ ]:
# 학습 결과 시각화
plt.figure('train', (12, 6))  # 그래프의 크기를 (12, 6)으로 설정하여 새로운 Figure 생성

# 에폭 평균 손실 그래프
plt.subplot(1, 2, 1)  # 서브 플롯 1 (1행 2열 중 첫 번째 위치)
plt.title("Epoch Average Loss")  # 그래프 제목 설정
x = [i + 1 for i in range(len(epoch_loss_history))]  # x축 값 (에폭 번호)
y = epoch_loss_history  # y축 값 (에폭 평균 손실)
plt.xlabel('epoch')  # x축 레이블 설정
plt.plot(x, y)  # 에폭에 따른 평균 손실 그래프 그리기

# 검증 AUC (Area under the ROC curve) 그래프
plt.subplot(1, 2, 2)  # 서브 플롯 2 (1행 2열 중 두 번째 위치)
plt.title("Validation: Area under the ROC curve")  # 그래프 제목 설정
x = [validation_interval * (i + 1) for i in range(len(metrics_history))]  # x축 값 (검증 간격에 따른 에폭 번호)
y = metrics_history  # y축 값 (AUC 값)
plt.xlabel('epoch')  # x축 레이블 설정
plt.plot(x, y)  # 에폭에 따른 AUC 값 그래프 그리기

# 그래프 표시
plt.show()  # 생성한 그래프들을 화면에 출력

#### 9. 테스트 데이터셋에서 모델 평가

In [ ]:
# 저장된 최적의 모델 로드 및 평가 설정
network.load_state_dict(torch.load('best_metric_model.pth'))
# 학습 시 저장된 최고 성능 모델의 가중치를 로드합니다.
network.eval()  # 모델을 평가 모드로 설정

# 테스트 데이터를 통해 예측 수행
y_true = list()  # 실제 레이블을 저장할 리스트
y_pred = list()  # 예측 레이블을 저장할 리스트
with torch.no_grad():  # 평가 시에는 기울기를 계산하지 않음
   for test_data in test_loader:
       test_images, test_labels = test_data[0].to(training_device), test_data[1].to(training_device)
       # 배치 데이터를 장치로 전송
       pred = network(test_images).argmax(dim=1)
       # 모델에 입력 데이터를 전달하여 각 이미지에 대한 예측 클래스 인덱스를 계산
       for i in range(len(pred)):
           y_true.append(test_labels[i].item())  # 실제 레이블을 리스트에 추가
           y_pred.append(pred[i].item())  # 예측된 레이블을 리스트에 추가

# 분류 보고서 출력
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred, target_names=category_names, digits=4))
# 실제 레이블과 예측 레이블을 비교하여 분류 성능 보고서를 출력합니다.
# 각 클래스의 정확도, 정밀도, 재현율, F1 점수를 계산하여 표시하며, 결과를 소수점 4자리까지 출력합니다.